# Transform raw data into Time-series data

In [1]:
import pandas as pd

rides = pd.read_parquet('../data/transformed/validated_rides_2023_01.parquet')
rides.head()

,pickup_datetime,pickup_location_id
0,2023-01-01 00:32:10,161
1,2023-01-01 00:55:08,43
2,2023-01-01 00:25:04,48
3,2023-01-01 00:03:48,138
4,2023-01-01 00:10:29,107


In [2]:
rides['pickup_hour']= rides['pickup_datetime'].dt.floor('H')
rides

/var/folders/d3/2wypd2852wd53tbd_4qqhq7h0000gn/T/ipykernel_79029/3002549705.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  rides['pickup_hour']= rides['pickup_datetime'].dt.floor('H')


,pickup_datetime,pickup_location_id,pickup_hour
0,2023-01-01 00:32:10,161,2023-01-01 00:00:00
1,2023-01-01 00:55:08,43,2023-01-01 00:00:00
2,2023-01-01 00:25:04,48,2023-01-01 00:00:00
3,2023-01-01 00:03:48,138,2023-01-01 00:00:00
4,2023-01-01 00:10:29,107,2023-01-01 00:00:00
...,...,...,...
3066761,2023-01-31 23:58:34,107,2023-01-31 23:00:00
3066762,2023-01-31 23:31:09,112,2023-01-31 23:00:00
3066763,2023-01-31 23:01:05,114,2023-01-31 23:00:00
3066764,2023-01-31 23:40:00,230,2023-01-31 23:00:00


In [3]:
agg_rides = rides.groupby(['pickup_hour','pickup_location_id']).size().reset_index()
agg_rides.rename(columns={0: 'rides'}, inplace=True)
agg_rides

,pickup_hour,pickup_location_id,rides
0,2023-01-01 00:00:00,4,19
1,2023-01-01 00:00:00,7,3
2,2023-01-01 00:00:00,12,1
3,2023-01-01 00:00:00,13,14
4,2023-01-01 00:00:00,24,20
...,...,...,...
71486,2023-01-31 23:00:00,261,5
71487,2023-01-31 23:00:00,262,11
71488,2023-01-31 23:00:00,263,41
71489,2023-01-31 23:00:00,264,40


In [4]:
from tqdm import tqdm

def add_missing_slots(agg_rides: pd.DataFrame) -> pd.DataFrame:
    """
    """
    location_ids = agg_rides['pickup_location_id'].unique()
    full_range = pd.date_range(
        agg_rides['pickup_hour'].min(), agg_rides['pickup_hour'].max(), freq='H')
    output = pd.DataFrame()
    for location_id in tqdm(location_ids):
        
        # keep only rides for this 'location_id'
        agg_rides_i = agg_rides.loc[agg_rides.pickup_location_id == location_id, ['pickup_hour','rides']]
        
        # quick way to add missing dates with 0 in a Series
        # taken from https://stackoverflow.com/a/19324591
        agg_rides_i.set_index('pickup_hour', inplace=True)
        agg_rides_i.index = pd.DatetimeIndex(agg_rides_i.index)
        agg_rides_i = agg_rides_i.reindex(full_range, fill_value=0)
        
        # add back 'location_id' columns
        agg_rides_i['pickup_location_id'] = location_id
        
        output = pd.concat([output, agg_rides_i])
        
    # move the purchase day from the index to a dataframe column
    output = output.reset_index().rename(columns={'index':'pickup_hour'})
    return output

In [5]:
agg_rides_all_slots = add_missing_slots(agg_rides)
agg_rides_all_slots

/var/folders/d3/2wypd2852wd53tbd_4qqhq7h0000gn/T/ipykernel_79029/1136877052.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  full_range = pd.date_range(
100%|████████████████████████████████████████| 257/257 [00:01<00:00, 232.60it/s]


,pickup_hour,rides,pickup_location_id
0,2023-01-01 00:00:00,19,4
1,2023-01-01 01:00:00,28,4
2,2023-01-01 02:00:00,43,4
3,2023-01-01 03:00:00,33,4
4,2023-01-01 04:00:00,12,4
...,...,...,...
191203,2023-01-31 19:00:00,0,2
191204,2023-01-31 20:00:00,0,2
191205,2023-01-31 21:00:00,0,2
191206,2023-01-31 22:00:00,0,2


In [8]:
from typing import Optional, List
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

def plot_rides(
    rides: pd.DataFrame,
    locations: Optional[List[int]] = None
):
    """
    Plot time-series data
    """
    rides_to_plot = rides[rides.pickup_location_id.isin(locations)] if locations else rides
    
    fig = px.line(
        rides_to_plot,
        x="pickup_hour",
        y="rides",
        color="pickup_location_id",
        template='none'
    )
    
    fig.show()

In [10]:
plot_rides(agg_rides_all_slots, locations=[43])

In [11]:
agg_rides_all_slots.to_parquet('../data/transformed/ts_data_2023_01.parquet')